In [1]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
API_key = 'JTFthLtGYuKFQK47MPvQcrabXJRJPk5nUEgaric4f17oP5VJcHyHqsqCMWXu'

#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    try:
        result = requests.get(api_call)
        result = result.json()
    except ValueError:
        print(f"Failed to decode JSON. Please check API call: {api_call}")

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False

    try:
        rate_limit = result['rate_limit']['remaining']
        reset = result['rate_limit']['resets_in_seconds']
    except:
        print('no rate limit available')

    return result, has_more, rate_limit, reset

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

def run_pages(api_first, api_second): 
    results, has_more, rate_limit, reset = run_api(f"{api_first}{api_second}")

    page = 2
    while has_more == True:
        if int(rate_limit) > 1:
            next_results, has_more, rate_limit, reset = run_api(f"{api_first}page={page}&{api_second}")
            results['data'] = results['data'] + next_results['data']
            page += 1
        else:
            print(f'Starting sleep timer for: {reset} seconds...')
            time.sleep(int(reset))
    else:
        pass
        
    return results

### **Load data**
<br>
Need to find which teams have played for which seasons, and then set the time interval to the season duration and only get the data for the correct teams.

In [3]:
teams_df = pd.read_csv('data/teams.csv')
seasons_df = pd.read_csv('data/seasons.csv')
leagues_df = pd.read_csv('data/leagues.csv')

Creating dictionary with seasons start date, end date, id, and teams participating in the season

In [48]:
season_dict = {'id':[], 'starting_at':[], 'ending_at':[]}
for row in seasons_df.index:
    app_dictionary('id', str(seasons_df.iloc[row]['id']), season_dict)
    app_dictionary('starting_at', str(seasons_df.iloc[row]['starting_at']), season_dict)
    app_dictionary('ending_at', seasons_df.iloc[row]['ending_at'], season_dict)
    app_dictionary('teams', [i for i in teams_df.loc[teams_df['season_id'] == int(seasons_df.iloc[row]['id'])]['id']], season_dict)


In [54]:
season_dict = pd.DataFrame.from_dict(season_dict)
season_dict.to_csv('data/seasons_info.csv')
season_dict.head()

,id,starting_at,ending_at,teams
0,19734,2022-08-05,2023-05-28,"[78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 2..."
1,19744,2022-08-05,2023-05-27,"[3317, 1079, 510, 3321, 3543, 68, 82, 3320, 67..."
2,19799,2022-08-12,2023-06-04,"[3477, 3468, 485, 36, 1099, 361, 6827, 645, 23..."
3,19806,2022-08-13,2023-06-11,"[10722, 2930, 397, 522, 1123, 625, 345, 1628, ..."
4,21646,2023-08-11,2024-05-19,"[78, 11, 8, 19, 29, 115, 51, 21, 14, 236, 52, ..."


In [72]:
start_dates = season_dict['starting_at']
end_dates = season_dict['ending_at']
teams_inSeasons = season_dict['teams']
season_id = season_dict['id']

# This allows you to work with all rows at once
for season_start, season_end, teams_inSeason, season_id in zip(start_dates, end_dates, teams_inSeasons, season_id):
    for team in teams_inSeason:
        start_date = datetime.strptime(season_start, '%Y-%m-%d')
        end_date = datetime.strptime(season_end, '%Y-%m-%d')
        #find parameters
        total_days = (end_date - start_date).days
        days_interval = 100

        # Variable to store the previous date
        previous_date = None
        # Loop over the 300 days in 100-day intervals\n",
        for i in range(0, total_days, days_interval):
            current_date = start_date + timedelta(days=i)
            formatted_date = current_date.strftime('%Y-%m-%d')

            # Print the previous and current date,
            if previous_date:
                print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
                first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + str(previous_date) + '/' + str(formatted_date) + '/' + {int(team)} + '?'
                second = 'api_token=' + API_key + '&per_page=50&select=name&include=metadata;formations;weatherReport:description,current;referees;events:type_id,addition,minute&filters=eventTypes:10,19,20,21;metadataTypes:567' 
                results = run_pages(first, second)
                try:
                    for fixture in results['data']:
                        fixture_data['data'].append(fixture)
                except:
                    print(f'No data for team {id}')
            else: # Avoiding calling only the first date 
                print(f"First date: {formatted_date}") 


296
295
296
302
282
289
274
288
246
253
253
246
282
274
282
274
275
267
275
281
352
316
338
344
281
267
281
275
275
267
275
274
283
281
274
274
274
267
273
268
267
274
267
274
267
267
287
274


In [110]:
first = 'https://api.sportmonks.com/v3/football/fixtures/between/2022-08-05/2022-09-10/78?'
second = 'api_token=' + API_key + '&per_page=50&select=name&include=metadata;formations;weatherReport:description,current;referees;events:type_id,addition,minute&filters=eventTypes:10,19,20,21;metadataTypes:567'
results = run_pages(first,second)
events = []
weather_pitch = {'fixture_id':[], 'pitch': [], 'weatherDescription':[], 'current':[]}
referee = {}
formation = []
for result in results['data']:
    events = events + result['events']
    weather_pitch['fixture_id'].append(result['id'])
    weather_pitch['pitch'].append(result['metadata']['values']['pitch'])
    weather_pitch['weatherDescription'].append(result['weatherreport']['description'])
    weather_pitch['weatherDescription'].append(result['weatherreport']['current'])



In [112]:
results

{'data': [{'name': 'Manchester United vs Brighton & Hove Albion',
   'id': 18535049,
   'sport_id': 1,
   'round_id': 274668,
   'stage_id': 77457864,
   'group_id': None,
   'aggregate_id': None,
   'league_id': 8,
   'season_id': 19734,
   'venue_id': 206,
   'state_id': 5,
   'starting_at_timestamp': None,
   'metadata': [{'id': 1157793,
     'metadatable_id': 18535049,
     'type_id': 567,
     'value_type': 'object',
     'values': {'pitch': 'GOOD'}}],
   'formations': [{'id': 336844,
     'fixture_id': 18535049,
     'participant_id': 78,
     'formation': '3-4-2-1',
     'location': 'away'},
    {'id': 336843,
     'fixture_id': 18535049,
     'participant_id': 14,
     'formation': '4-2-3-1',
     'location': 'home'}],
   'weatherreport': {'id': 103325,
    'fixture_id': 18535049,
    'venue_id': 206,
    'description': 'broken clouds',
    'current': None},
   'referees': [{'id': 3014813,
     'fixture_id': 18535049,
     'referee_id': 13537,
     'type_id': 9},
    {'id': 301